In [3]:
import pandas as pd
import pymysql

# Connect to the database
connection = pymysql.connect(
    host='127.0.0.1',
    user='root',
    password='abhi12345',
    database='sakila'
)

# Execute the SQL query and read into a Pandas DataFrame
query = '''
   SELECT
    f.film_id,
    f.title,
    f.release_year,
    f.length,
    f.rating,
    f.special_features,
    COUNT(r.rental_id) AS rental_count,
    MAX(IFNULL(r.rental_date >= '2005-05-01' AND r.rental_date < '2005-06-01', 0)) AS rented_in_may
FROM
    film f
LEFT JOIN
    inventory i ON f.film_id = i.film_id
LEFT JOIN
    rental r ON i.inventory_id = r.inventory_id
GROUP BY
    f.film_id;

'''
df = pd.read_sql(query, connection)

# Close the database connection
connection.close()


In [4]:
# Encoding categorical variables (example)
df = pd.get_dummies(df, columns=['rating', 'special_features'])

# Scaling numerical variables (example)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[['release_year', 'length', 'rental_count']] = scaler.fit_transform(df[['release_year', 'length', 'rental_count']])

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Define features (X) and target variable (y)
X = df.drop(['film_id', 'title', 'rented_in_may'], axis=1)
y = df['rented_in_may']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{class_report}')


Accuracy: 0.81
Confusion Matrix:
[[ 39  26]
 [ 12 123]]
Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.60      0.67        65
           1       0.83      0.91      0.87       135

    accuracy                           0.81       200
   macro avg       0.80      0.76      0.77       200
weighted avg       0.81      0.81      0.80       200

